# **Project 4: PointCloud-LLM (PC-LLM)**

In this project, you will learn how to adapt a large language model (LLM) to handle **non-text inputs**, specifically **3D point clouds**. The objective is to enable the LLM to generate **human-like descriptions** for input 3D point clouds.

The project is similar to [PointLLM](https://arxiv.org/abs/2308.16911) with totally different LLM and heavily optimized code.

The dataset consists of approximately **500K descriptions** from the [Cap3D](https://arxiv.org/abs/2306.07279) dataset.

---

## **Learning Objectives**

By the end of this project, you will understand how to:

- Project a non-text modality (3D point clouds) into the LLM embedding space.  
- Introduce and train **special tokens** within the LLM.  
- Construct **multimodal prompts** using the latest **Hugging Face** interface.  
- Fine-tune the model with **LoRA (Low-Rank Adaptation)** using the `peft` library.  
- Save and load LoRA fine-tuned models.  
- Explore different **token generation strategies**, including greedy decoding and top-p sampling.


**Requirement** ⚠️: Please implement all the TODOS!
---

## **Model Architecture**

The overall architecture is illustrated below:  
![img](PC_LLM.png)

We will use a **LLaMA 3.2 (1B)** model as the language backbone, combined with a **pretrained PointBERT** encoder for processing point cloud inputs.

For simplicity, the **point cloud feature** will be represented as a **single token**, obtained by **max pooling** the PointBERT 512 embedding vectors and concatenating the max pooled vector with the CLS vector.

---

## **Training Setup**

- **Hardware requirements:**  
  - 1 × GPU with **24 GB VRAM** (e.g., RTX 3090)  
  - **10 GB** CPU RAM  

- **Training time:** ~**2 hours 35 minutes**  
- **Hyperparameter tuning:** Not required if the implementation is correct — a single training run should suffice.

---

## **Resources**

## The following resources are required and can be downloaded [here](https://drive.google.com/drive/folders/1XCJhVgT_YQ5ocwFEBGIudSVFxlV0UX5T?usp=sharing) 🌍:  
- LLaMA 3.2 (1B) model  
- Pretrained PointBERT encoder  
- Cap3D dataset
- Shapes colored Pointclouds


# Preparing the environment
The code is tested on cuda 11.8, pytorch 2.5.1 (please make sure you are using the exact version), and Python 3.10.19
We assume at least cuda 11.7 is installed. 

```bash
conda create -n llama32 python=3.10 -y
conda activate llama32

pip install torch==2.5.1 torchvision==0.20.1 torchaudio==2.5.1 --index-url https://download.pytorch.org/whl/cu118
pip install transformers==4.46.2 accelerate==0.33.0 safetensors==0.4.5 sentencepiece==0.2.0\
pip install bitsandbytes==0.43.2
pip install huggingface_hub==0.24.6
pip install omegaconf timm matplotlib termcolor tqdm peft plotly gradio
conda install jupyter
```

# Extracting the point cloud

```
cat Objaverse_660K_8192_npy_split_a* > Objaverse_660K_8192_npy.tar.gz
tar -xvf Objaverse_660K_8192_npy.tar.gz
```

# Other files/folders
- `PointLLM_brief_description_val_3000_GT.json`: will be the test set
- `pointllm_500k_dataset.json`: will be the training set
- `point_bert_v1.2.pt` is the pretrained PointBert
- `Llama-3.2-1B-Instruct` contains LLama3.2 1B model weights


In [173]:
import torch 
print(torch.cuda.device_count()) 
for i in range(torch.cuda.device_count()):
    print(f"GPU {i}: {torch.cuda.get_device_name(i)}")

1
GPU 0: NVIDIA A100-SXM4-80GB


In [174]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0" # Please use your appropriate gpu id

In [175]:
import math
import random
import json
import time
import numpy as np
from dataclasses import dataclass
from tqdm.auto import tqdm
from typing import List, Dict, Any

import torch
import torch.nn as nn
from pointbert.point_encoder import *
from omegaconf import OmegaConf

from torch.utils.data import Dataset, DataLoader
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    get_linear_schedule_with_warmup
)

In [176]:
if not torch.cuda.is_available():
    raise ValueError("Please make sure the used pytorch version runs on the gpu")

In [177]:
# ------------------------------------------------------------
# Config
# ------------------------------------------------------------
MODEL_ID = "/ibex/user/zharkyy/P4_lab/Llama-3.2-1B-Instruct" 
point_bert_checkpoint_path = "/ibex/user/zharkyy/P4_lab/point_bert_v1.2.pt"
DEVICE = "cuda:0"

# Used for optimization pruporses to make the code fit inside a single gpu of 24GB
DTYPE = torch.bfloat16 

THREE_D_DIM = 768 # The 3D feature dimension
BATCH_SIZE = 32 # The batch size
EPOCHS = 1 # We will train for a single epoch
MAX_LEN = 384 # The maximum allowed context length of the LLM

In [178]:
# ------------------------------------------------------------
# Tokenizer, Model & Adding the special tokens
# ------------------------------------------------------------
tokenizer = AutoTokenizer.from_pretrained(MODEL_ID)
pad_token_id = tokenizer.eos_token_id
tokenizer.pad_token_id = tokenizer.eos_token_id

print(f"Tokenizer vocab initial size: {len(tokenizer)}")
newly_added_tokens_count = 0
POINT_CLOUD_TOKEN = "<POINT_CLOUD>"
POINT_START_TOKEN = "<POINT_START>"
POINT_END_TOKEN = "<POINT_END>"

# TODO write your code here
new_tokens = {
    'additional_special_tokens': [
        POINT_CLOUD_TOKEN,
        POINT_START_TOKEN,
        POINT_END_TOKEN
    ]
}
newly_added_tokens_count = tokenizer.add_special_tokens(new_tokens)

# Sanity check
assert newly_added_tokens_count == 3
print(f"Tokenizer vocab size after adding 3 special tokens: {len(tokenizer)}")

Tokenizer vocab initial size: 128256
Tokenizer vocab size after adding 3 special tokens: 128259


## Constructing the LLM Training Prompt

This section explains how to build the input prompt for the LLM (LLaMA-based) during training.

First please have a look [here](https://huggingface.co/docs/transformers/en/chat_templating).

```python
model_test_messages = [
     {"role": "system", "content": "You are a helpful assistant that describes 3D shapes."},
     {"role": "user", "content": f"Please provide a caption for the input shape: {POINT_START_TOKEN}{POINT_CLOUD_TOKEN}{POINT_END_TOKEN}"},
]

model_train_messages = model_test_messages + [
    {"role": "assistant", "content": "<<<CAPTION>>>"}
]
```

### Key Guidelines

1. **Model Objective**
   - The LLM should **predict only the shape caption tokens**.
   - Only the caption tokens should be marked as ground-truth (`label`) tokens.
   - All **non-caption tokens**, including padding, should be **masked with `-100`**.

2. **Prompt Construction**
   - Use `self.tok.apply_chat_template()` and `self.tok()` to handle tokenization and chat-style formatting.
   - Enable **truncation** and set the **maximum context length** to `MAX_LEN`.

3. **Two-Stage Tokenization Strategy**
   - **First call:**  
     - Generate tokens for the input text *without* the shape caption.  
     - Use this to determine the number of input (non-labels) tokens.  
     - Set `add_generation_prompt=True` to avoid predicting `"ASSISTANT:"` tokens part.
   - **Second call:**  
     - Generate tokens for the **full conversation** (including the shape caption).  
     - Set `add_generation_prompt=False` since the assistant message (caption) is already part of the input.

4. **Point Cloud Feature Integration**
   - During the training/inference, the embedding of the `POINT_CLOUD_TOKEN` (at index `pc_token_pos`) will be replaced with the **shape point cloud feature** extracted from **PointBERT**. `POINT_CLOUD_TOKEN` is just used as a placeholder.

---

> **Tip:** Following this process ensures that the LLaMA model only learns to generate descriptive captions for point clouds, without being confused by instruction or role tokens. Please have a look at the construct_prompt for the test part for hints. 


In [179]:
# ------------------------------------------------------------
# Dataset class and visualize the point cloud
# ------------------------------------------------------------
class ShapeCaptionDataset(Dataset):
    def __init__(self, pointclouds_dir, captions_file_path, tokenizer, pointcloud_token, max_len=MAX_LEN, mode='train', debug=None):
        super().__init__()
        
        # Load the point clouds
        self.pointclouds_path = pointclouds_dir
        self.tok = tokenizer
        self.mode = mode
        self.max_len = max_len
        self.pointcloud_token_id = tokenizer.convert_tokens_to_ids(pointcloud_token)

        # Load the annotations
        self.captions = []
        with open(captions_file_path) as fin:
            captions = json.load(fin)
        
        filter_ids = ['6760e543e1d645d5aaacd3803bcae524', 'b91c0711149d460a8004f9c06d3b7f38'] # Bad examples without color needs to be excluded
        
        for data in captions:
            object_id = data['object_id']
            if object_id in filter_ids:
                continue
            self.captions.append(data)

        if debug is not None:
            self.captions = self.captions[:debug]

    def __len__(self): 
        return len(self.captions)
    
    def pc_norm(self, pc):
        """ 
        Center the point cloud around the origin point scaled to fit inside a unit sphere
        pc: Nx6 (XYZ RGB)
        Return the normalized pc Nx6 
        """
        # TODO write your code here
        # pc = None
        xyz = pc[:, :3]
        
        centroid = np.mean(xyz, axis=0)
        xyz = xyz - centroid
        
        m = np.max(np.sqrt(np.sum(xyz**2, axis=1)))
        xyz = xyz / m
        
        pc[:, :3] = xyz
        
        return pc

    def construct_prompt(self, caption):
        if self.mode == 'train':
            messages = [
                {"role": "system", "content": "You are a helpful assistant that describes 3D shapes."},
                {"role": "user", "content": f"Please provide a caption for the input shape: {POINT_START_TOKEN}{POINT_CLOUD_TOKEN}{POINT_END_TOKEN}"},
                {"role": "assistant", "content": caption} # Add the shape caption
            ]
        elif self.mode == 'test':
            messages = [
                {"role": "system", "content": "You are a helpful assistant that describes 3D shapes."},
                {"role": "user", "content": f"Please provide a caption for the input shape: {POINT_START_TOKEN}{POINT_CLOUD_TOKEN}{POINT_END_TOKEN}"},
            ]
        else:
            raise ValueError

        if self.mode == "train":
            # TODO write your code here
            messages_without_caption = messages[:-1]  
            prompt_without_caption = self.tok.apply_chat_template(
                messages_without_caption, 
                tokenize=False, 
                add_generation_prompt=True
            )
            enc_without_caption = self.tok(
                prompt_without_caption, 
                return_tensors="pt", 
                truncation=True, 
                max_length=self.max_len
            )
            num_input_tokens = enc_without_caption['input_ids'].shape[1]
            
            prompt_with_caption = self.tok.apply_chat_template(
                messages, 
                tokenize=False, 
                add_generation_prompt=False
            )
            enc_with_caption = self.tok(
                prompt_with_caption, 
                return_tensors="pt", 
                truncation=True, 
                max_length=self.max_len
            )
            
            input_ids = enc_with_caption['input_ids'].squeeze(0)
            attention_mask = enc_with_caption['attention_mask'].squeeze(0)
            
            labels = input_ids.clone()
            labels[:num_input_tokens] = -100  # Mask input tokens
            labels[input_ids == self.tok.pad_token_id] = -100  # Mask padding tokens
            
            pc_token_pos = (input_ids == self.pointcloud_token_id).nonzero(as_tuple=True)[0]
            pc_token_pos = pc_token_pos.item() if len(pc_token_pos) > 0 else -1
            
            # pc_token_pos is the index of the POINT_CLOUD_TOKEN inside the prompt, what do you think should it be the same for all descriptions?
            # No, it can vary because different prompts may have different lengths. However, in our case with a fixed prompt, the position should be the same for all samples.
            return input_ids, attention_mask, labels, pc_token_pos
            
        elif self.mode == 'test':
            # Note that there should be no labels here as this is the inference mode
            prompt = self.tok.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
            enc = tokenizer(prompt, return_tensors="pt", truncation=True, max_length=self.max_len)
            
            # TODO write your code here
            input_ids = enc['input_ids'].squeeze(0)
            attention_mask = enc['attention_mask'].squeeze(0)
            
            # Find the position of POINT_CLOUD_TOKEN
            pc_token_pos = (input_ids == self.pointcloud_token_id).nonzero(as_tuple=True)[0]
            pc_token_pos = pc_token_pos.item() if len(pc_token_pos) > 0 else -1
            
            labels = [] # Keep it as is. we do not need to provide labels during the inference
            
            return input_ids, attention_mask, labels, pc_token_pos
    
    def __getitem__(self, idx): 
        data = self.captions[idx]
        
        # Get the object id
        object_id = data['object_id']
        
        # Get the caption
        caption = data['conversations'][-1]['value']
        
        # Get the colored point cloud
        filename = f"{object_id}_8192.npy"    
        point_cloud = np.load(os.path.join(self.pointclouds_path, filename))
        point_cloud = self.pc_norm(point_cloud) # * need to norm since point encoder is norm

        input_ids, attention_mask, labels, pc_token_pos = self.construct_prompt(caption)
                
        return dict(pointcloud=point_cloud, 
                    caption=caption, 
                    object_id=object_id, 
                    input_ids=input_ids, 
                    attention_mask=attention_mask, 
                    labels=labels, 
                    pc_token_pos=pc_token_pos)

In [180]:
@dataclass
class Batch:
    input_ids: torch.Tensor
    attention_mask: torch.Tensor
    labels: torch.Tensor
    obj_positions: torch.Tensor
    pointclouds: torch.Tensor
    gt_captions: list

class Collator:
    def __init__(self):
        pass

    def __call__(self, items: List[Dict[str, Any]]) -> Batch:
        input_ids, attention_mask, labels, obj_positions, gt_captions = [], [], [], [], []
        pointclouds = [] 

        for example_data in items:
            input_ids.append(example_data['input_ids'])
            attention_mask.append(example_data['attention_mask'])
            labels.append(example_data['labels'])
            obj_positions.append(example_data['pc_token_pos'])
            gt_captions.append(example_data["caption"])
            pointclouds.append(torch.from_numpy(example_data['pointcloud']).float())



        # Stack into batch with the necessary padding if needed 
        # TODO write your code here, have a look at the nn.utils.rnn.pad_sequence fucntion

        input_ids = torch.nn.utils.rnn.pad_sequence(input_ids, batch_first=True, padding_value=tokenizer.pad_token_id)
        attention_mask = torch.nn.utils.rnn.pad_sequence(attention_mask, batch_first=True, padding_value=0)

        
        # labels = torch.nn.utils.rnn.pad_sequence(labels, batch_first=True, padding_value=-100)
        # pointclouds = torch.stack(pointclouds, dim=0)
        if labels and isinstance(labels[0], torch.Tensor):
            labels = torch.nn.utils.rnn.pad_sequence(labels, batch_first=True, padding_value=-100)
        else:
            labels = []  # Keep as empty list for test mode
        
        pointclouds = torch.stack(pointclouds, dim=0)

        return Batch(
            input_ids=input_ids,
            attention_mask=attention_mask,
            labels=labels,
            obj_positions=torch.tensor(obj_positions, dtype=torch.long),
            pointclouds=pointclouds,
            gt_captions=gt_captions
        )
        
collator = Collator()

In [181]:
# Create the training dataset
train_ds = ShapeCaptionDataset(
    pointclouds_dir="/ibex/user/zharkyy/P4_lab/8192_npy",
    captions_file_path="/ibex/user/zharkyy/P4_lab/pointllm_500k_dataset.json",
    tokenizer=tokenizer, 
    pointcloud_token=POINT_CLOUD_TOKEN,
    mode='train',
    debug=None # You can can write a small number (like 500 training example) for debugging
)

print(f"The len of the training dataset is {len(train_ds)}")

test_ds = ShapeCaptionDataset(
    pointclouds_dir="/ibex/user/zharkyy/P4_lab/8192_npy",
    captions_file_path="/ibex/user/zharkyy/P4_lab/PointLLM_brief_description_val_3000_GT.json",
    tokenizer=tokenizer, 
    pointcloud_token=POINT_CLOUD_TOKEN,
    mode='test',
)

print(f"The len of the test dataset is {len(test_ds)}")

train_loader = DataLoader(train_ds, batch_size=BATCH_SIZE, shuffle=True, collate_fn=collator, drop_last=True) 
test_loader = DataLoader(test_ds, batch_size=1, shuffle=False, collate_fn=collator, drop_last=False)

The len of the training dataset is 499998
The len of the test dataset is 3000


In [182]:
# Create the model
model = AutoModelForCausalLM.from_pretrained(
    MODEL_ID,
    # torch_dtype="auto",
    dtype = "auto",
    device_map="auto"
)

# Resize the input and output embedding layers to accommodate the three newly added tokens
# Please have a look here: https://huggingface.co/docs/transformers/main_classes/model#transformers.PreTrainedModel.resize_token_embeddings
# to see how the tokens are initialized? please make sure you are reading the most recent docuementation.
# Please read this https://www.cs.columbia.edu/~johnhew/vocab-expansion.html
# TODO write your code here
model.resize_token_embeddings(len(tokenizer), mean_resizing=False)

# Freeze all LLM params
# Use simple loop; i.e. for p in model.parameters()
# TODO write your code here
for p in model.parameters():
    p.requires_grad = False
model.get_input_embeddings().weight[-3:].requires_grad = True
print(f"Model embedding size: {model.get_input_embeddings().weight.shape}")
print(f"Trainable parameters in embeddings: {model.get_input_embeddings().weight[-3:].numel()}")

Model embedding size: torch.Size([128259, 2048])
Trainable parameters in embeddings: 6144


In [ ]:
# Construct the LoRA Config 
from peft import LoraConfig, get_peft_model, TaskType
from peft import get_peft_model, PeftModel

lora_config = LoraConfig(
    r=8,                           # Rank of the LoRA matrices
    lora_alpha=32,                 # Scaling factor
    target_modules=["q_proj", "v_proj"],
    lora_dropout=0.05,             # Dropout on LoRA layers
    bias="none",
    task_type= TaskType.CAUSAL_LM # TODO+
)

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


In [172]:
# Create the peft model 
model = get_peft_model(model, lora_config)
model.train()
print("***** LoRA trainable parameters *****")
model.print_trainable_parameters()

# make sure that also the input embedding layers are trainable, useful function model.get_input_embeddings()

# What is the output embedding layer? 
# The output embedding layer shares weights with input embeddings in LLaMA

# Do we need to modify the weights of the output embedding layer?
# Since we've already unfrozen the new token embeddings in input_embeddings, output embedding layer will also have gradients for those tokens
# We don't need to separately modify lm_head as it typically shares weights with input embeddings

# TODO write your code here
model.get_input_embeddings().weight[-3:].requires_grad = True


print("***** The full trainable parameters *****") # should be higher 
model.print_trainable_parameters()

***** LoRA trainable parameters *****
trainable params: 851,968 || all params: 1,236,672,512 || trainable%: 0.0689
***** The full trainable parameters *****
trainable params: 851,968 || all params: 1,236,672,512 || trainable%: 0.0689


/ibex/user/zharkyy/conda-environments/llama32/lib/python3.10/site-packages/peft/mapping_func.py:72: UserWarning:

You are trying to modify a model with PEFT for a second time. If you want to reload the model with a different config, make sure to call `.unload()` before.



In [154]:
model.to(DEVICE)
print("Moved model to GPU")

Moved model to GPU


In [155]:
# TODO write your code here
llm_hidden_dim = model.get_input_embeddings().weight.shape[1]
print(f"LLM hidden dimension: {llm_hidden_dim}")
################################################################
#####################    3D Projector   ########################
################################################################
class Projector(nn.Module):
    def __init__(self, in_dim: int, out_dim: int, width: int = 1024):
        super().__init__()

        # 2 linear layers with hidden dim of 1024 and use one GELU between the two linear layers
        # TODO write your code here
        self.net = nn.Sequential(
            nn.Linear(in_dim, width),
            nn.GELU(),
            nn.Linear(width, out_dim)
        )
        
    def forward(self, x):
        return self.net(x)

projector = Projector(THREE_D_DIM, llm_hidden_dim).to(DEVICE).to(DTYPE)

LLM hidden dimension: 2048


In [156]:
################################################################
######################     PointBert   ######################### 
################################################################
class PointEncoder(nn.Module):
    def __init__(self, config_path: str="pointbert_config.yaml", flant5_hidden_dim: int = 2048):
        super(PointEncoder, self).__init__()
        
        # Read the config
        point_bert_config = OmegaConf.load(config_path)
        point_bert_config.model.point_dims = 6
        use_max_pool = True
        
        self.point_backbone = PointTransformer(point_bert_config.model, use_max_pool=use_max_pool)
        self.point_backbone_config = {
            "point_cloud_dim": point_bert_config.model.point_dims,
            "backbone_output_dim": point_bert_config.model.trans_dim if not use_max_pool else point_bert_config.model.trans_dim * 2,
            "project_output_dim": flant5_hidden_dim,
            "point_token_len": point_bert_config.model.num_group + 1 if not use_max_pool else 1, # * number of output features, with cls token
            "projection_hidden_layer": point_bert_config.model.get('projection_hidden_layer', 0),
            "use_max_pool": use_max_pool
        }
        
        if point_bert_config.model.get('projection_hidden_layer', 0) > 0:
            self.point_backbone_config["projection_hidden_dim"] = point_bert_config.model.projection_hidden_dim 
        
    def load_point_backbone_checkpoint(self, checkpoint_path=None):
        self.point_backbone.load_checkpoint(checkpoint_path)
        
    def forward(self, point_clouds):
        point_features = self.point_backbone(point_clouds)
        return point_features

In [157]:
# Load the pretrained checkpoint
point_encoder = PointEncoder().to(DEVICE).to(DTYPE)
point_encoder.load_point_backbone_checkpoint(point_bert_checkpoint_path)
point_encoder.eval() # do not forget to convert it to evaluation mode

print("Created and loaded the point encoder")

/ibex/user/zharkyy/P4_lab/pointbert/point_encoder.py:145: FutureWarning:

You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.

2025-12-07 15:03:44,069 - Transformer - INFO - PointBERT's weights ar

Created and loaded the point encoder


In [158]:
# ------------------------------------------------------------
# Training setup
# ------------------------------------------------------------
optimizer = torch.optim.AdamW([
    {"params": model.parameters(), "lr": 0.0002, "weight_decay": 0.00},   # model
    {"params": projector.parameters(), "lr": 0.001, "weight_decay": 0.0} # projector
])

num_steps = EPOCHS * math.ceil(len(train_loader))
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=max(1, num_steps // 20), num_training_steps=num_steps)
scaler = torch.cuda.amp.GradScaler(enabled=False)

/tmp/ipykernel_1005666/518217905.py:11: FutureWarning:

`torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.



In [159]:
# ------------------------------------------------------------
# Training loop
# ------------------------------------------------------------
for epoch in range(EPOCHS):
    assert model.training and projector.training
    
    for step, batch in tqdm(enumerate(train_loader), total=len(train_loader), desc='Trainig'):
        optimizer.zero_grad(set_to_none=True)

        # Ship to cuda
        input_ids = batch.input_ids.to(DEVICE)
        attention_mask = batch.attention_mask.to(DEVICE)
        labels = batch.labels.to(DEVICE)
        obj_positions = batch.obj_positions.to(DEVICE)
        pointclouds = batch.pointclouds.to(DEVICE).to(DTYPE)

        # Get the token embeddings
        base_embeds = model.get_input_embeddings()(input_ids)

        # Get the point cloud feature 
        with torch.no_grad():
            shape_feats = point_encoder(pointclouds).squeeze(1)

        # Project the point cloud feature 
        with torch.autocast(device_type="cuda", dtype=DTYPE):
            proj_vec = projector(shape_feats)

        # Replace the point cloud feature for each example with the POINT_CLOUD_TOKEN placeholder embedding
        for b in range(base_embeds.size(0)):
            pos = obj_positions[b].item()
            
            if 0 <= pos < base_embeds.size(1):
                # TODO write your code here.
                base_embeds[b, pos, :] = proj_vec[b]

            else:
                raise ValueError("there is no <POINT_CLOUD_TOKEN> token in the input!")

        # Predict the output
        with torch.autocast(device_type="cuda", dtype=DTYPE):
            out = model(inputs_embeds=base_embeds.cuda(), attention_mask=attention_mask.cuda(), labels=labels.cuda())
            loss = out.loss

        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()
        scheduler.step()

        if step % 250 == 0:
            print(f"Epoch {epoch} | Step {str(step).zfill(4)} | Loss {loss.item():.4f}")

print("Training done...")

Trainig:   0%|          | 0/15624 [00:00<?, ?it/s]

Epoch 0 | Step 0000 | Loss 3.7458
Epoch 0 | Step 0250 | Loss 1.6609
Epoch 0 | Step 0500 | Loss 1.6463
Epoch 0 | Step 0750 | Loss 1.7157
Epoch 0 | Step 1000 | Loss 1.6277
Epoch 0 | Step 1250 | Loss 1.6395
Epoch 0 | Step 1500 | Loss 1.5725
Epoch 0 | Step 1750 | Loss 1.4899
Epoch 0 | Step 2000 | Loss 1.4850
Epoch 0 | Step 2250 | Loss 1.6715
Epoch 0 | Step 2500 | Loss 1.4016
Epoch 0 | Step 2750 | Loss 1.2402
Epoch 0 | Step 3000 | Loss 1.2801
Epoch 0 | Step 3250 | Loss 1.4786
Epoch 0 | Step 3500 | Loss 1.2130
Epoch 0 | Step 3750 | Loss 1.3339
Epoch 0 | Step 4000 | Loss 1.2528
Epoch 0 | Step 4250 | Loss 1.3375
Epoch 0 | Step 4500 | Loss 1.3838
Epoch 0 | Step 4750 | Loss 1.5100
Epoch 0 | Step 5000 | Loss 1.2210
Epoch 0 | Step 5250 | Loss 1.2865
Epoch 0 | Step 5500 | Loss 1.2775
Epoch 0 | Step 5750 | Loss 1.2869
Epoch 0 | Step 6000 | Loss 1.3548
Epoch 0 | Step 6250 | Loss 1.3973
Epoch 0 | Step 6500 | Loss 1.2057
Epoch 0 | Step 6750 | Loss 1.4344
Epoch 0 | Step 7000 | Loss 1.1937
Epoch 0 | Step

In [168]:
# ------------------------------------------------------------
# Saving the model and the projector 
# ------------------------------------------------------------

# Save the projector weights (done for you)
projector_save_path = f"projector_weights_{len(train_ds)}.pth"
torch.save(projector.state_dict(), projector_save_path)
print(f"Projector weights saved to {projector_save_path}")    

# Save the model weights
# Merge LoRA adapters into the base model, use merge_and_unload and save_pretrained functions
# Do we need to save also the tokenizer?
# Yes, I need to save the tokenizer because it contains the special tokens that i added, 
# and inference requires the same tokenizer to properly encode prompts and decode outputs.
merged_model_save_dir = "my_full_finetuned_model"

assert isinstance(model, PeftModel)

# TODO write your code here
merged_model = model.merge_and_unload()
merged_model.save_pretrained(merged_model_save_dir)
tokenizer.save_pretrained(merged_model_save_dir)

print(f"Merged model saved to {merged_model_save_dir}")
print(f"Tokenizer saved to {merged_model_save_dir}")


Projector weights saved to projector_weights_499998.pth
Merged model saved to my_full_finetuned_model
Tokenizer saved to my_full_finetuned_model


In [161]:
# ------------------------------------------------------------
# load saved checkpoints
# ------------------------------------------------------------
# TODO write your code here. For the model use AutoModelForCausalLM.from_pretrained, what about the tokenizer?
loaded_model = AutoModelForCausalLM.from_pretrained(merged_model_save_dir)

loaded_tokenizer = AutoTokenizer.from_pretrained(merged_model_save_dir)
loaded_projector = Projector(THREE_D_DIM, llm_hidden_dim)
loaded_projector.load_state_dict(torch.load(projector_save_path))
loaded_projector.to(DEVICE).to(DTYPE)
loaded_projector.eval()

print("Model, tokenizer, and projector loaded")

The tokenizer you are loading from 'my_full_finetuned_model' with an incorrect regex pattern: https://huggingface.co/mistralai/Mistral-Small-3.1-24B-Instruct-2503/discussions/84#69121093e8b480e709447d5e. This will lead to incorrect tokenization. You should set the `fix_mistral_regex=True` flag when loading this tokenizer to fix this issue.


Model, tokenizer, and projector loaded


/tmp/ipykernel_1005666/1692920574.py:9: FutureWarning:

You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.



In [162]:
import gradio as gr
import torch
import plotly.graph_objs as go

# ------------------------------------------------------------
# Helper: visualize a single point cloud using Plotly
# ------------------------------------------------------------
def visualize_pointcloud(points):
    """
    Visualize a 3D colored point cloud (Nx6 tensor or array: [x,y,z,r,g,b])
    """
    xyz = points[:, :3]
    rgb = points[:, 3:]

    # Normalize color if it's in [0,255]
    if rgb.max() > 1.0:
        rgb = rgb / 255.0

    # Convert to 0–255 range for Plotly
    colors = (rgb * 255).astype(int)
    color_strings = [
        f"rgb({r},{g},{b})" for r, g, b in colors
    ]

    fig = go.Figure(
        data=[go.Scatter3d(
            x=xyz[:, 0],
            y=xyz[:, 1],
            z=xyz[:, 2],
            mode='markers',
            marker=dict(
                size=3,
                opacity=0.9,
                color=color_strings
            )
        )]
    )

    fig.update_layout(
        margin=dict(l=0, r=0, t=0, b=0),
        scene=dict(
            xaxis_title='X',
            yaxis_title='Y',
            zaxis_title='Z',
            aspectmode='data'
        ),
        showlegend=False
    )

    return fig

# ------------------------------------------------------------
# Caption generation wrapper for Gradio
# ------------------------------------------------------------
@torch.no_grad()
def generate_caption(base, shape_feats, input_ids, obj_positions, method, max_new_tokens=128) -> str:
    obj_pos = obj_positions[0]
    proj = projector(shape_feats)
    base[0, obj_pos, :] = proj[0]
    attention_mask=torch.ones(base.shape[:-1], dtype=torch.long, device=DEVICE)

    if method == 'greedy':
        # TODO write your code here
        gen_ids = model.generate(
           inputs_embeds=base,
            attention_mask=attention_mask,
            max_new_tokens=max_new_tokens,
            do_sample=False
        )
    elif method == 'top-p':
        # TODO write your code here
        # use adequate parameters
        gen_ids = model.generate(
           inputs_embeds=base,
            attention_mask=attention_mask,
            max_new_tokens=max_new_tokens,
            do_sample=True,
            top_p=0.9,
            temperature=0.7
        )
    else:
        raise ValueError("Invalid generation method")

    # return tokenizer.decode(gen_ids[0], skip_special_tokens=True)
    if isinstance(gen_ids, list):
        gen_ids = torch.tensor(gen_ids)
    return tokenizer.decode(gen_ids[0], skip_special_tokens=True)

# ------------------------------------------------------------
# Function to run inference for one batch from test_loader
# ------------------------------------------------------------
def run_inference(sample_idx=0, method='greedy', not_gradio=False):
    model.eval()
    projector.eval()

    for step, batch in enumerate(test_loader):
        if step == sample_idx:
            input_ids = batch.input_ids.to(DEVICE)
            attention_mask = batch.attention_mask.to(DEVICE)
            obj_positions = batch.obj_positions.to(DEVICE)
            pointclouds = batch.pointclouds.to(DEVICE).to(DTYPE)
            gt_caption = batch.gt_captions

            with torch.no_grad():
                base_embeds = model.get_input_embeddings()(input_ids)
                shape_feats = point_encoder(pointclouds).to(DEVICE).to(DTYPE)
                gen_caption = generate_caption(base_embeds, shape_feats, input_ids, obj_positions, method=method)

            if not_gradio:
                return gen_caption, gt_caption
            else:
                pc_fig = visualize_pointcloud(pointclouds[0].detach().cpu().float().numpy())
                
            return pc_fig, gen_caption, gt_caption
    return None, "No sample found", ""

# ------------------------------------------------------------
# Gradio UI
# ------------------------------------------------------------
demo = gr.Interface(
    fn=run_inference,
    inputs=[
        gr.Slider(0, 500, value=0, step=1, label="Sample index"), # Showing only the first 500 shapes 
        gr.Radio(["greedy", "top-p"], value="greedy", label="Generation Method")
    ],
    outputs=[
        gr.Plot(label="Point Cloud"),
        gr.Textbox(label="Generated Caption"),
        gr.Textbox(label="Ground Truth Caption")
    ],
    title="3D Point Cloud Captioning Demo",
    description="Visualize a 3D point cloud from the test loader and generate captions using a pretrained model."
)

demo.launch(share=True, inline=True)


* Running on local URL:  http://127.0.0.1:7863
* Running on public URL: https://23e2bc8b08f7994019.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


In [163]:
demo = gr.Interface(
    fn=run_inference,
    inputs=[
        gr.Slider(0, 500, value=0, step=1, label="Sample index"), # Showing only the first 500 shapes 
        gr.Radio(["greedy", "top-p"], value="greedy", label="Generation Method")
    ],
    outputs=[
        gr.Plot(label="Point Cloud"),
        gr.Textbox(label="Generated Caption"),
        gr.Textbox(label="Ground Truth Caption")
    ],
    title="3D Point Cloud Captioning Demo",
    description="Visualize a 3D point cloud from the test loader and generate captions using a pretrained model."
)

demo.launch(share=True, inline=True)


* Running on local URL:  http://127.0.0.1:7864
* Running on public URL: https://ebe2a19c5167b6261e.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


In [164]:
demo = gr.Interface(
    fn=run_inference,
    inputs=[
        gr.Slider(0, 500, value=0, step=1, label="Sample index"), # Showing only the first 500 shapes 
        gr.Radio(["greedy", "top-p"], value="greedy", label="Generation Method")
    ],
    outputs=[
        gr.Plot(label="Point Cloud"),
        gr.Textbox(label="Generated Caption"),
        gr.Textbox(label="Ground Truth Caption")
    ],
    title="3D Point Cloud Captioning Demo",
    description="Visualize a 3D point cloud from the test loader and generate captions using a pretrained model."
)

demo.launch(share=True, inline=True)


* Running on local URL:  http://127.0.0.1:7865
* Running on public URL: https://2448e1edd5c264aa77.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


In [165]:
demo = gr.Interface(
    fn=run_inference,
    inputs=[
        gr.Slider(0, 500, value=0, step=1, label="Sample index"), # Showing only the first 500 shapes 
        gr.Radio(["greedy", "top-p"], value="greedy", label="Generation Method")
    ],
    outputs=[
        gr.Plot(label="Point Cloud"),
        gr.Textbox(label="Generated Caption"),
        gr.Textbox(label="Ground Truth Caption")
    ],
    title="3D Point Cloud Captioning Demo",
    description="Visualize a 3D point cloud from the test loader and generate captions using a pretrained model."
)

demo.launch(share=True, inline=True)

* Running on local URL:  http://127.0.0.1:7866
* Running on public URL: https://9843af59408faacedf.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


In [184]:
import pandas as pd

# saving greedy predictions, please make sure the csv files are saved and the cell output is shown
tuples = []
for i in range(100):
    tuples.append(run_inference(sample_idx=i, method='greedy', not_gradio=True))

greedy_preds = pd.DataFrame(tuples)
greedy_preds.columns = ["generated_caption", "GT_caption"]
greedy_preds.to_csv("greedy_preds.csv")
greedy_preds

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for

,generated_caption,GT_caption
0,"Here are a few options:\n\n1. ""The trusty old ...",[An electrical weighing machine with a white c...
1,Here are three different captions for the shap...,[College walls]
2,Here are three different captions for the shap...,[Grey coloured bat toy.]
3,Here's a possible caption for the input shape ...,[White thin panel with red edge]
4,"""Red sports car, sleek and powerful, with a cu...",[Carro rojo y negro con cuatro llantas negras]
...,...,...
95,"""Police Car: 2018 Ford F-150 Police Intercepto...",[A police car with a siren on the to p and hav...
96,Here's a possible caption for the 3D shape: \n...,[A white building structure with two parts and...
97,Here are three different captions for the shap...,[A cartoon girl with red eyes]
98,"Here's a caption for the shape ""attract"":\n\n""...",[Red coloured toy dinosaur model.]


In [167]:
# Saving top-p predictions
tuples = []
for i in range(100):
    tuples.append(run_inference(sample_idx=i, method='top-p', not_gradio=True))

# saving top-p predictions, please make sure the csv files are saved and the cell output is shown
top_p_preds = pd.DataFrame(tuples)
top_p_preds.columns = ["generated_caption", "GT_caption"]
top_p_preds.to_csv("top_p_preds.csv")
top_p_preds

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for

,generated_caption,GT_caption
0,A 3D model of a white lamp with a cord and plu...,[An electrical weighing machine with a white c...
1,A 3D model of a torn-apart house with graffiti...,[College walls]
2,"A 3D model featuring a bird, frog, lizard, and...",[Grey coloured bat toy.]
3,A 3D rendering of a wooden door with a white f...,[White thin panel with red edge]
4,A 3D model of a red sports car.,[Carro rojo y negro con cuatro llantas negras]
...,...,...
95,3D model of a police car with a white roof and...,[A police car with a siren on the to p and hav...
96,A 3D model of a building featuring yellow and ...,[A white building structure with two parts and...
97,3D model of a woman in a yellow dress flying w...,[A cartoon girl with red eyes]
98,A 3D model of a horse with a long tail and horns.,[Red coloured toy dinosaur model.]


In [185]:
print(greedy_preds)

                                    generated_caption                                         GT_caption
0   Here are a few options:\n\n1. "The trusty old ...  [An electrical weighing machine with a white c...
1   Here are three different captions for the shap...                                    [College walls]
2   Here are three different captions for the shap...                           [Grey coloured bat toy.]
3   Here's a possible caption for the input shape ...                   [White thin panel with red edge]
4   "Red sports car, sleek and powerful, with a cu...     [Carro rojo y negro con cuatro llantas negras]
..                                                ...                                                ...
95  "Police Car: 2018 Ford F-150 Police Intercepto...  [A police car with a siren on the to p and hav...
96  Here's a possible caption for the 3D shape: \n...  [A white building structure with two parts and...
97  Here are three different captions for the shap...  

In [186]:
print(top_p_preds)

                                    generated_caption                                         GT_caption
0   A 3D model of a white lamp with a cord and plu...  [An electrical weighing machine with a white c...
1   A 3D model of a torn-apart house with graffiti...                                    [College walls]
2   A 3D model featuring a bird, frog, lizard, and...                           [Grey coloured bat toy.]
3   A 3D rendering of a wooden door with a white f...                   [White thin panel with red edge]
4                     A 3D model of a red sports car.     [Carro rojo y negro con cuatro llantas negras]
..                                                ...                                                ...
95  3D model of a police car with a white roof and...  [A police car with a siren on the to p and hav...
96  A 3D model of a building featuring yellow and ...  [A white building structure with two parts and...
97  3D model of a woman in a yellow dress flying w...  